This is MLP method

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [2]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()

# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
# 104 gene for training
train_lst=random.sample(gene_num_list, 104)#####
# 26 gene for testing
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)

# filter out compound        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']

# 80% target gene of Compound + Gene perturbation 
dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    

# To meet the mAP evaluation Plate
# 2x2 : Time_delay x Cell_line
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])

train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))
train_part_total_final=train_part_total_final.drop_duplicates()


# SEEN test
df_test1=pd.concat([gene_seen,dftrain2,df_negcon]) 
df_test1=df_test1[df_test1['Metadata_Plate'].isin(lst)]
# Train 
df_train=pd.concat([train_part_total_final,dftrain2]) 

### Unseen test
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=pd.concat([df_test2,df_negcon,dftrain2])
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_Plate'].isin(lst)]

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test1.drop('Metadata_gene', axis = 1)
y_test = df_test1['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_309656/2760319005.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)
# Train the MLPClassifier model
mlp_clf = MLPClassifier(hidden_layer_sizes=(1280, 630, 300),
                        max_iter=300, activation='relu', verbose=True, tol=1e-8,
                        solver='adam')
mlp_clf.fit(trainX_scaled, trainY)

Iteration 1, loss = 4.27615198
Iteration 2, loss = 3.22013432
Iteration 3, loss = 2.59175545
Iteration 4, loss = 2.10432153
Iteration 5, loss = 1.70370076
Iteration 6, loss = 1.40945760
Iteration 7, loss = 1.02898154
Iteration 8, loss = 0.75414133
Iteration 9, loss = 0.62825756
Iteration 10, loss = 0.48866785
Iteration 11, loss = 0.42590878
Iteration 12, loss = 0.27344329
Iteration 13, loss = 0.16354673
Iteration 14, loss = 0.13181838
Iteration 15, loss = 0.07126101
Iteration 16, loss = 0.04094628
Iteration 17, loss = 0.02905724
Iteration 18, loss = 0.01850291
Iteration 19, loss = 0.01330847
Iteration 20, loss = 0.01219409
Iteration 21, loss = 0.01179927
Iteration 22, loss = 0.00960775
Iteration 23, loss = 0.00896286
Iteration 24, loss = 0.00678872
Iteration 25, loss = 0.00502463
Iteration 26, loss = 0.00878172
Iteration 27, loss = 0.00753003
Iteration 28, loss = 0.00407521
Iteration 29, loss = 0.00352943
Iteration 30, loss = 0.00328185
Iteration 31, loss = 0.00310136
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(1280, 630, 300), max_iter=300, tol=1e-08,
              verbose=True)

In [4]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test1.iloc[:,df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test1.iloc[:,0:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test1['Metadata_experiment_type']], axis=1)

In [5]:
feature_new.to_csv('MLP_seen_new.csv')
df_test1.to_csv('MLP_seen_old.csv')    

In [6]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test2.iloc[:,df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test2.iloc[:,0:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test2['Metadata_experiment_type']], axis=1)

In [7]:
feature_new.to_csv('MLP_unseen_new.csv')
df_test2.to_csv('MLP_unseen_old.csv')  

This is SLPP Method

In [8]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']

# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    

# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])

train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))



# SEEN Test

df_test1=pd.concat([gene_seen,dftrain2,df_negcon]) 
df_test1=df_test1[df_test1['Metadata_Plate'].isin(lst)]

# Train
df_train=pd.concat([train_part_total_final,dftrain2])


### Unseen test
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=pd.concat([df_test2,df_negcon,dftrain2])
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_Plate'].isin(lst)]

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
y_test=df_test1['Metadata_gene']
testX=df_test1.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_309656/703226900.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [9]:
X_train = trainX
y_train = y_train.reset_index()
y_train=y_train['Metadata_gene'].tolist()
n_components=300
sigma=1.0

In [10]:
    # Standardize the data
    X_train = StandardScaler().fit_transform(X_train)
    # Compute the pairwise distance matrix
    D = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            D[i, j] = np.linalg.norm(X_train[i] - X_train[j])

    # Compute the adjacency graph using a Gaussian kernel
    W = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            W[i, j] = np.exp(-D[i, j] ** 2 / (2 * sigma ** 2)) * (y_train[i] == y_train[j])

    # Compute the degree matrix
    D = np.diag(np.sum(W, axis=1))

    # Compute the Laplacian matrix
    L = D - W

    # Compute the eigenvectors
    eigvals, eigvecs = np.linalg.eig(np.dot(np.dot(X_train.T, L), X_train))

    # Sort the eigenvectors by eigenvalues in descending order
    sort_indices = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, sort_indices]

    # Select the first n_components eigenvectors
    eigvecs = eigvecs[:, :n_components]
    eigvecs=np.real(eigvecs)

In [11]:
## transformed seen_test_data
transformed_data = np.dot(testX, eigvecs)
new_feature=pd.DataFrame(transformed_data)
testX1=df_test1.iloc[:,:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX1=testX1.reset_index()
dfnew =pd.concat([testX1,new_feature],axis=1)

In [12]:
dfnew.iloc[:,1:].to_csv('SLPP_seen_new.csv')          # After feature transformation
df_test1.to_csv('SLPP_seen_old.csv')                  # Before feature transformation

In [13]:
# Train_test pre-process Unseen test data
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
y_test=df_test2['Metadata_gene']
testX=df_test2.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
X_train = trainX
y_train = y_train.reset_index()
y_train=y_train['Metadata_gene'].tolist()

In [14]:
# Transformed Unseen test Data
transformed_data = np.dot(testX, eigvecs)
new_feature=pd.DataFrame(transformed_data)
testX2=df_test2.iloc[:,:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX2=testX2.reset_index()
dfnew =pd.concat([testX2,new_feature],axis=1)

In [15]:
dfnew.iloc[:,1:].to_csv('SLPP_unseen_new.csv')
df_test1.to_csv('SLPP_unseen_old.csv')    